 Objective: Train a model to detect fraudulent providers with suspiciously high claim volumes in short durations using inpatient + outpatient data

In [ ]:
# import libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# load outpatient data 
# Replace these paths with the correct file paths on your machine
df_fraud = pd.read_csv("/Users/kalibraun/dev/healthcare/fraudTrain.csv.xls")
df_in = pd.read_csv("/Users/kalibraun/dev/healthcare/inpatientData.csv")
df_out = pd.read_csv("/Users/kalibraun/dev/healthcare/outpatientData.csv")

In [ ]:
# look at df_fraud, should be yes and no in potentialfraud column
print(df_fraud)

In [ ]:
print(df_in.columns)
print(df_out.columns)

In [ ]:
# convert df_fraud from bool to int
df_fraud["PotentialFraud"] = df_fraud["PotentialFraud"].map({"Yes": 1, "No": 0})
print(df_fraud)

Preprocessing and Feature Engineering

In [ ]:
# create claim duration column
# convert date columns to datetime format 
df_in["ClaimStartDt"] = pd.to_datetime(df_in["ClaimStartDt"])
df_in["ClaimEndDt"] = pd.to_datetime(df_in["ClaimEndDt"])

df_out["ClaimStartDt"] = pd.to_datetime(df_out["ClaimStartDt"])
df_out["ClaimEndDt"] = pd.to_datetime(df_out["ClaimEndDt"])

# create new duration column in days
df_in["ClaimDuration"] = (df_in["ClaimEndDt"] - df_in["ClaimStartDt"]).dt.days
df_out["ClaimDuration"] = (df_out["ClaimEndDt"] - df_out["ClaimStartDt"]).dt.days

In [ ]:
# aggregate inpatient data by provider 
in_agg = df_in.groupby("Provider").agg(
    InpatientClaimCount=("Provider", "count"),
    InpatientTotalReimbursed=("InscClaimAmtReimbursed", "sum"),
    InpatientAvgDuration=("ClaimDuration", "mean")
).reset_index()

# Aggregate outpatient data by provider
out_agg = df_out.groupby("Provider").agg(
    OutpatientClaimCount=("Provider", "count"),
    OutpatientTotalReimbursed=("InscClaimAmtReimbursed", "sum"),
    OutpatientAvgDuration=("ClaimDuration", "mean")
).reset_index()

# Merge inpatient + outpatient
provider_features = pd.merge(in_agg, out_agg, on="Provider", how="outer").fillna(0)

# Merge with fraud labels
df_fraud["PotentialFraud"] = df_fraud["PotentialFraud"]
df = pd.merge(provider_features, df_fraud, on="Provider", how="inner")

In [ ]:
print(df.head())

Train Test Split

In [ ]:
features = [
    'InpatientClaimCount', 'InpatientTotalReimbursed', 'InpatientAvgDuration',
    'OutpatientClaimCount', 'OutpatientTotalReimbursed', 'OutpatientAvgDuration'
]
X = df[features]
y = df["PotentialFraud"]

In [ ]:
from xgboost import XGBClassifier

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=42)
xgb_clf.fit(X_train, y_train)

print("XGBoost Accuracy:", xgb_clf.score(X_test, y_test))

Evaluate Model

In [ ]:
# confusion matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

y_pred = xgb_clf.predict(X_test)

cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Not Fraud", "Fraud"])
disp.plot(cmap="Blues", values_format="d")
plt.title("Confusion Matrix")
plt.show()

print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=["Not Fraud", "Fraud"]))

Visualizations

In [ ]:
# Visualize Feature Importance
importances = pd.Series(xgb_clf.feature_importances_, index=features)

plt.figure(figsize=(8, 6))
importances.sort_values().plot(kind="barh", color="steelblue")
plt.title("XGBoost Feature Importances")
plt.xlabel("Importance")
plt.show()

In [ ]:
# Scatter Plot: Claim Volume vs Fraud Probability
sns.set(style = "whitegrid")

# Get fraud probabilities from the model
df["FraudProbability"] = xgb_clf.predict_proba(X)[:, 1]  # Probability of class 1 (fraud)

# create totalcaimcount column
df["TotalClaimCount"] = df["InpatientClaimCount"] + df["OutpatientClaimCount"]

# Create a scatter plot
plt.figure(figsize=(10, 6))
sns.scatterplot(
    data=df,
    x= "TotalClaimCount",  
    y="FraudProbability",
    hue="PotentialFraud",
    palette={0: "green", 1: "red"}, # geen = no fraud, red = fraud
    alpha=0.6
)

plt.title("Claim Volume vs. Predicted Fraud Probability")
plt.xlabel("Total Claim Count (Inpatient + Outpatient)")
plt.ylabel("Predicted Fraud Probability")
plt.legend(title="Actual Fraud")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# highlight top N providers with highest fraud probability 

# Get top N riskiest providers
top_n = 10
top_providers = df.sort_values("FraudProbability", ascending=False).head(top_n)

# Base scatter plot
plt.figure(figsize=(12, 7))
sns.scatterplot(
    data=df,
    x="TotalClaimCount",
    y="FraudProbability",
    hue="PotentialFraud",
    palette={0: "green", 1: "red"},
    alpha=0.5,
    legend=False
)

# Highlight top N
sns.scatterplot(
    data=top_providers,
    x="TotalClaimCount",
    y="FraudProbability",
    color="black",
    s=100,
    marker="X",
    label=f"Top {top_n} Risky Providers"
)

# Annotate provider IDs
for _, row in top_providers.iterrows():
    plt.text(
        row["TotalClaimCount"] + 1,  # Slight offset
        row["FraudProbability"],
        row["Provider"],
        fontsize=9,
        color="black"
    )

plt.title("Claim Volume vs. Predicted Fraud Probability (Top Risky Providers Highlighted)")
plt.xlabel("Total Claim Count")
plt.ylabel("Predicted Fraud Probability")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


Black "X" markers show the highest-risk providers.
Their IDs are annotated for clarity.
You can change top_n to highlight more/less risky ones.

In [ ]:
# plotly Version: Interactive Scatter Plot
import plotly.express as px

# Make sure TotalClaimCount is calculated
df["TotalClaimCount"] = df["InpatientClaimCount"] + df["OutpatientClaimCount"]

# Optional: Label fraud class as string for prettier legend
df["FraudLabel"] = df["PotentialFraud"].map({0: "Not Fraud", 1: "Fraud"})

# Create interactive scatter plot
fig = px.scatter(
    df,
    x="TotalClaimCount",
    y="FraudProbability",
    color="FraudLabel",
    hover_data=["Provider", "TotalClaimCount", "FraudProbability"],
    title="Interactive: Claim Volume vs Predicted Fraud Probability",
    labels={"TotalClaimCount": "Total Claims", "FraudProbability": "Fraud Probability"},
    color_discrete_map={"Fraud": "red", "Not Fraud": "green"},
    opacity=0.6
)

# Highlight top N risky providers
top_n = 10
top_providers = df.nlargest(top_n, "FraudProbability")

fig.add_scatter(
    x=top_providers["TotalClaimCount"],
    y=top_providers["FraudProbability"],
    mode="markers+text",
    marker=dict(size=12, color="black", symbol="x"),
    text=top_providers["Provider"],
    textposition="top center",
    name=f"Top {top_n} Risky Providers"
)

fig.update_layout(legend_title_text="Actual Fraud Label", height=600)

fig.show()


Ways to improving model 
- Feature Engineering 
- Class Weights or Sampling
- Hyperparameter Tuning
- More Evaluation Metrics
- Model Stacking / Blending
- SHAP or Feature Importance

In [ ]:
# shap - This can reveal what makes providers look suspicious 
import shap

explainer = shap.TreeExplainer(xgb_clf)
shap_values = explainer.shap_values(X)

shap.summary_plot(shap_values, X)

In [ ]:
# more evaluation metrics
from sklearn.metrics import classification_report, roc_auc_score

y_proba = xgb_clf.predict_proba(X_test)[:, 1] # predicted probabilities

print(classification_report(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_proba))

In [ ]:
# roc curve plot
# the closer the curve is to the top left corner, the better your model is
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# Get false positive rate, true positive rate, thresholds
fpr, tpr, thresholds = roc_curve(y_test, y_proba)

# Calculate AUC
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f"ROC Curve (AUC = {roc_auc:.2f})", color='darkorange', linewidth=2)
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')  # Diagonal baseline
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver Operating Characteristic (ROC) Curve")
plt.legend(loc="lower right")
plt.grid(True)
plt.show()
